### Ingestion de la carpeta "production_country"

### Paso 1 - Leer los archivos JSON usando "DataFrameReader" de Spark

In [0]:
dbutils.widgets.text("p_enviroment", "")
v_enviroment = dbutils.widgets.get("p_enviroment")

In [0]:
dbutils.widgets.text("p_file_date", "2024-12-16")
v_file_date = dbutils.widgets.get("p_file_date")

In [0]:
%run "../includes/configuration"

In [0]:
%run "../includes/common_functions"

In [0]:
from pyspark.sql.functions import current_timestamp,lit,col
from pyspark.sql.types import StructType,StructField,IntegerType


In [0]:
productions_countries_schema = StructType([
    StructField("movieId", IntegerType(), True),
    StructField("countryId", IntegerType(), True)
    ])


In [0]:
productions_countries_df = spark.read.schema(productions_countries_schema).option("multiLine", True).json(f"{bronze_folder_path}/{v_file_date}/production_country/")#Activo con el option la lectura multilinea del json

In [0]:
productions_countries_df.printSchema()

In [0]:
display(productions_countries_df)

#### Paso 2 - Renombrar las columnas y añadir nuevas columnas
1. "movieId" renombrar a "movie_id"
2. "countryId" renombrar a "country_id"
3. Agregar las columnas "ingestion_date" y "enviroment"

In [0]:
productions_countries_final_df = add_ingestion_date(productions_countries_df).withColumnsRenamed({"movieId":"movie_id","countryId":"country_id"}).withColumns({"enviroment":lit(v_enviroment),"file_date":lit(v_file_date)})

In [0]:
display(productions_countries_final_df)

#### Paso 3 - Escribir la salida en formato "Parquet"

In [0]:
merge_condition = 'tgt.movie_id = src.movie_id AND tgt.country_id = src.country_id AND tgt.file_date = src.file_date'
merge_delta_lake(productions_countries_final_df, 'movie_silver', 'productions_countries', silver_folder_path, merge_condition, 'file_date')

In [0]:
%fs
ls /mnt/historialpeliculas/silver/productions_countries/

In [0]:
%sql
SELECT file_date, COUNT(1)
FROM movie_silver.productions_countries
GROUP BY file_date;

In [0]:
dbutils.notebook.exit("Success")